# Wie Sie dieses Notebook nutzen:
- Führen Sie diesen Code Zelle für Zelle aus.
- Um die Variableninhalte zu beobachten, nutzen Sie in Jupyter-Classic den "Variable Inspektor". Falls Sie dieses Notebook in Jupyter-Lab verwenden, nutzen Sie hierfür den eingebauten Debugger.
- Wenn Sie "Code Tutor" zur Visualisierung des Ablaufes einzelner Zellen nutzen möchten, führen Sie einmalig die nachfolgende Zelle aus. Anschliessend schreiben Sie %%tutor in die erste Zeile jeder Zelle, die Sie visualisieren möchten.
- Die Dokumentation von [range()](https://docs.python.org/3/library/stdtypes.html?highlight=range#range), [len()](https://docs.python.org/3/library/functions.html?highlight=len#len) und allen anderen eingebauten Funktionen finden Sie auf [docs.python.org](https://docs.python.org/3/library/functions.html)
- Der [Spickzettel](https://rocco.melzian.ch/archiv/www.jython.ch/download/spickzettel.pdf) leistet vielfach gute Dienste.

In [ ]:
# Für Code Tutor Visualisierungen
from metakernel import register_ipython_magics
register_ipython_magics()

In [ ]:
# Für im Docstring eingebettete Tests
from doctest import run_docstring_examples

# Für Erzeugung von Zufalls-Spielfeldern
import random

### Rekursion - eine Anwendung

#### Aufgabenstellung
Sie möchten auf einem $n$ x $m$ Spielfeld vom Startfeld $S$ zum Zielfeld $Z$ laufen und dabei möglichst viele Punkte sammeln. Sie dürfen sich in jedem Zug nur je ein Feld nach rechts oder ein Feld abwärts bewegen. Diagonale Bewegungen sind nicht erlaubt.

Jedes Feld enhält eine Ihnen vorab bekannte Punktzahl ( > 0 ), die Sie beim Betreten einsammeln können.

Was ist die maximale Punktzahl welche Sie erzielen können, wenn Sie auf folgendem $m$ x $n$ Spielfeld von $S=(0,0)$ nach $Z(m-1,n-1)$ gehen?

#### Beispiel
S = (0,0) links oben, Z = (3,3) rechts unten

<table>
<tr>
<td></td>
<td>j</td>
</tr>
<tr>
<td>i</td>
<td>
<table>
<tr>
<td><b><i>1</i></b></td>
<td><b>3</b></td>
<td><b>4</b></td>
<td>1</td>
</tr>
<tr>
<td>2</td>
<td>3</td>
<td><b>6</b></td>
<td><b>8</b></td>
</tr>
<tr>
<td>1</td>
<td>2</td>
<td>2</td>
<td><b>3</b></td>
</tr>
<tr>
<td>4</td>
<td>3</td>
<td>2</td>
<td><b><i>2</i></b></td>
</tr>
</table>
</td>
</tr>
</table>


Lösung: 1 + 3 + 4 + 6 + 8 + 3 + 2 = 27

In [ ]:
DEMO_MATRIX_0 = [
    [1]
]

DEMO_MATRIX_1 = [
    [0, 3],
    [4, 0]
]

DEMO_MATRIX_2 = [
    [1, 3, 3],
    [1, 1, 1],
    [1, 4, 1]
]

DEMO_MATRIX_3 = [
    [1, 3, 4, 1],
    [2, 3, 6, 8],
    [1, 2, 2, 3],
    [4, 3, 2, 2]
]

#### Implementierung mit SuS
##### Setup
- {1,4,9,16,...} Schülerinnen und Schüler stellen sich als $m$ x $n$ Matrix auf und
- blicken zur Tafel und bleiben in dieser Blickrichtung stehen und
- identifizieren den zu ihrer Position zugehörigen Feldwert in der Matrix und
- merken sich diesen Feldwert (oder notieren Sie diesen auf einem kleinen Spickzettel).

##### Algorithmus
Wenn Sie nun nach den Punkten auf ihrem Feld gefragt werden, dann:

- Berechnen Sie

$$meine\_punkte = mein\_feldwert + max(punkte\_oben, punkte\_links)$$

- Dazu fragen Sie zuerst die Person links neben Ihnen nach ihren Punkten und warten die Antwort ab.
- Dann fragen Sie die Person vor Ihnen und warten die Antwort ab.
- Wenn keine Person vor und/oder links neben Ihnen steht (Sie also auf dem Rand der Matrix stehen), nehmen Sie jeweils den Wert 0 als Punktzahl für die fehlende Person an.*** 
- Berechnen Sie nun die Summe Ihres Feldwertes plus dem Maximum der beiden Antworten, die Sie von vorn und links erhalten haben
- und antworten nun Ihrerseits mit dem Ergebnis ($meine\_punkte$) ihrer Berechnung.

*** Wenn Sie also bspw. auf Feld (i=0,j=0) stehen (ganz links oben), können Sie direkt mit ihrem Feldwert antworten, da niemand vor oder links neben ihnen steht!

##### Anwendung
Wenn L nun wissen will, wieviele Punkte maximal auf dem Weg von $S$ nach $Z$ einer Matrix gesammelt werden können, dann

- erzeugt L die Matrix und stellt diese an der Tafel dar und dann
- bittet L seine SuS, die Schritte aus dem Setup auszuführen und dann
- fragt L den/die SoS auf dem Zielfeld nach seiner Punktzahl und dann
- wartet L bis die Anwort gegeben wurde.

##### Erweiterung des Algorithmus
- merken Sie sich, wie oft Sie nach Ihren Punkten gefragt werden und
- nach Abschluss der Berechnung schreiben Sie diese Zahl in die leere Matrix an der Tafel.

##### Fragen an die beobachtenden Schülerinnen und Schüler
- Warum löst dieser simple Algorithmus ein so komplexes Problem? Jede Person kennt doch nur ihren eigenen Feldwert und führt lediglich eine extrem einfache Berechnung (meine_punkte = mein_feldwert + max(punkte_oben, punkte_links)) aus? Seltsam ... ?!!??
  * ???
- Ist der Algorithmus effizient im dem Sinne, das nichts doppelt berechnet wird?
  * ???
- Wie kann der Algorithmus verbessert werden?
  * ???
  * ???

#### Implementierung in Python

##### Rekursiver Lösungsansatz
- Gegeben: m x n - Matrix
- Abbruchbedingung:
    * am Start angekommen (i == j == 0 -> return Feldwert von Matrix[0][0])
    * oder ausserhalb des Spielfeldes (i < 0 oder j < 0 oder i >= m, j >= n -> return 0)
- Rekursionsgleichung:
    * f(i,j) = matrix(i, j) + max( f(i - 1, j), f(i, j - 1) )
    
##### Python-Implementierung

In [ ]:
def punkte(matrix, i=None, j=None):
    """
    Berechnet die maximal möglichen Punkte vom Start (i=0, j=0) bis zu den Zielkoordinaten in i und j.
    
    Tests:
    
    >>> print(punkte(DEMO_MATRIX_0))
    1

    >>> print(punkte(DEMO_MATRIX_1))
    4

    >>> print(punkte(DEMO_MATRIX_2))
    10

    >>> print(punkte(DEMO_MATRIX_3))
    27

    """
    m = len(matrix)
    n = len(matrix[0])
    
    if i == None or j == None: # Default-Startposition ist "Rechts unten"
        i = m-1
        j = n-1
    
    if ??? :  # Abbruchbedinung: Feldrand erreicht?
        return 0

    feldwert = matrix[i][j]

    if ??? :  # Abbruchbedingung: i=0, j=0 
        return feldwert

    above = ???
    left = ???

    return feldwert + max(above, left)

# Testen
run_docstring_examples(punkte, locals())

##### Anwendung der Lösung

In [ ]:
m = 4
n = 4
matrix = list(random.choices(range(1,10), k=n) for _ in range(m)) # siehe https://docs.python.org/3/library/random.html?highlight=random.choices#random.choices

for zeile in matrix:
    print(zeile)

In [ ]:
msg = "Sie können maximal {} Punkte erreichen, wenn Sie durch dieses {} x {} Spielfeld gehen."
max_punkte = punkte(matrix)
print(msg.format(max_punkte, m, n))